## Utility Functions Class
helps in the process of solving and manipulating states

In [ ]:
class Grid:
    SIZE = 3
    GOAL = (1,2,3,4,5,6,7,8,0)

    @staticmethod
    def get_blank(state):
        """Return (row, col) of blank (0)."""
        index = state.index(0)
        return divmod(index, Grid.SIZE)

    @staticmethod
    def swap(state, r1, c1, r2, c2):
        """Return new state with two positions swapped."""
        lst = list(state)
        i1 = r1 * Grid.SIZE + c1
        i2 = r2 * Grid.SIZE + c2
        lst[i1], lst[i2] = lst[i2], lst[i1]
        return tuple(lst)

    @staticmethod
    def move(state, direction):
        """Return new state after move, or None if invalid."""
        r, c = Grid.get_blank(state)

        if direction == "up" and r > 0:
            return Grid.swap(state, r, c, r-1, c)

        if direction == "down" and r < Grid.SIZE - 1:
            return Grid.swap(state, r, c, r+1, c)

        if direction == "left" and c > 0:
            return Grid.swap(state, r, c, r, c-1)

        if direction == "right" and c < Grid.SIZE - 1:
            return Grid.swap(state, r, c, r, c+1)

        return None

    @staticmethod
    def neighbors(state):
        """Return list of all reachable states."""
        moves = ["up", "down", "left", "right"]
        result = []

        for m in moves:
            new_state = Grid.move(state, m)
            if new_state is not None:
                result.append(new_state)

        return result

    @staticmethod
    def manhattan(state):
        """Return Manhattan distance heuristic."""
        distance = 0

        for i, value in enumerate(state):
            if value == 0:
                continue

            goal_row, goal_col = divmod(value - 1, Grid.SIZE)
            curr_row, curr_col = divmod(i, Grid.SIZE)

            distance += abs(goal_row - curr_row) + abs(goal_col - curr_col)

        return distance

    @staticmethod
    def is_goal(state):
        """Returns True if the current state is a goal state False otherwise"""
        return state == Grid.GOAL
    
    @staticmethod
    def print_state(state):
        """Prints the state to the console in a 3x3 grid format"""
        for i in range(9):
            if i % 3 == 0 and i != 0:
                print()
            print(state[i] if state[i] != 0 else "_", end=" ")
        print() 

In [22]:
def is_solvable(state):
    """
    Return True if 8-puzzle state is solvable.
    """
    tiles = [x for x in state if x != 0]

    inversion_count = 0

    for i in range(len(tiles)):
        for j in range(i + 1, len(tiles)):
            if tiles[i] > tiles[j]:
                inversion_count += 1

    return inversion_count % 2 == 0


## Solver Function with A* Search

In [32]:
import heapq

def solve(problem):
    if not is_solvable(problem):
        return None
    
    open_list = []
    closed_list = {}
    solved = False

    heapq.heappush(open_list,(Grid.manhattan(problem),0,problem))

    while len(open_list) > 0 and not solved:
        f,pred,curr = heapq.heappop(open_list)

        # lazy deletion if we already processed this state earlier with a better eval func
        if curr in closed_list:
            continue
        
        closed_list[curr] = pred
        successors = Grid.neighbors(curr)
        for c in successors:
            if Grid.is_goal(c):
                solved = True
                closed_list[c] = curr

            if c not in closed_list:
                g = f - Grid.manhattan(curr)
                g_c = g + 1
                h_c = Grid.manhattan(c)
                f_c = g_c + h_c
                heapq.heappush(open_list,(f_c,curr,c))

    # backtrack to find the sequence of states 
    if solved:
        path = []
        curr = Grid.GOAL
        path.append(curr)
        while closed_list[curr] != 0:
            path.append(closed_list[curr])
            curr = closed_list[curr]

        path.reverse()

        return path

    else:
        return None


## Pygame Visualization

Interactive visualization of the solving process with controls:
- **SPACE**: Pause/Resume automatic playback
- **RIGHT ARROW**: Next step (manual mode)
- **LEFT ARROW**: Previous step (manual mode)
- **ESC**: Exit visualization

In [35]:
import pygame

def visualize_solution(path, delay=500):
    """
    Visualize the 8-puzzle solution using pygame.
    
    Args:
        path: List of states from initial to goal
        delay: Milliseconds between each step (default 500ms)
    """
    if path is None:
        print("No solution to visualize!")
        return
    
    # Initialize pygame
    pygame.init()
    
    # Constants
    WINDOW_SIZE = 450
    GRID_SIZE = 3
    TILE_SIZE = WINDOW_SIZE // GRID_SIZE
    MARGIN = 5
    
    # Colors
    BG_COLOR = (240, 240, 240)
    TILE_COLOR = (70, 130, 180)
    TEXT_COLOR = (255, 255, 255)
    BLANK_COLOR = (200, 200, 200)
    HEADER_BG = (50, 50, 50)
    HEADER_TEXT = (255, 255, 255)
    
    # Create window
    HEADER_HEIGHT = 60
    screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE + HEADER_HEIGHT))
    pygame.display.set_caption('8-Puzzle Solver Visualization')
    
    # Font
    font = pygame.font.Font(None, 60)
    header_font = pygame.font.Font(None, 36)
    
    def draw_state(state, step_num, total_steps):
        """Draw the current state of the puzzle."""
        screen.fill(HEADER_BG)
        
        # Draw header
        header_text = f"Step {step_num}/{total_steps - 1}  |  Manhattan: {Grid.manhattan(state)}"
        text_surface = header_font.render(header_text, True, HEADER_TEXT)
        text_rect = text_surface.get_rect(center=(WINDOW_SIZE // 2, HEADER_HEIGHT // 2))
        screen.blit(text_surface, text_rect)
        
        # Draw grid background
        grid_surface = pygame.Surface((WINDOW_SIZE, WINDOW_SIZE))
        grid_surface.fill(BG_COLOR)
        
        # Draw tiles
        for i in range(9):
            value = state[i]
            row = i // GRID_SIZE
            col = i % GRID_SIZE
            
            x = col * TILE_SIZE + MARGIN
            y = row * TILE_SIZE + MARGIN
            width = TILE_SIZE - 2 * MARGIN
            height = TILE_SIZE - 2 * MARGIN
            
            # Choose color based on whether it's blank or not
            if value == 0:
                tile_color = BLANK_COLOR
            else:
                tile_color = TILE_COLOR
            
            # Draw tile
            pygame.draw.rect(grid_surface, tile_color, (x, y, width, height), border_radius=10)
            
            # Draw number (skip for blank)
            if value != 0:
                text = font.render(str(value), True, TEXT_COLOR)
                text_rect = text.get_rect(center=(col * TILE_SIZE + TILE_SIZE // 2,
                                                   row * TILE_SIZE + TILE_SIZE // 2))
                grid_surface.blit(text, text_rect)
        
        screen.blit(grid_surface, (0, HEADER_HEIGHT))
        pygame.display.flip()
    
    # Main visualization loop
    current_step = 0
    clock = pygame.time.Clock()
    paused = False
    running = True
    auto_play = True
    last_step_time = pygame.time.get_ticks()
    
    print("\n=== Controls ===")
    print("SPACE: Pause/Resume")
    print("RIGHT ARROW: Next step")
    print("LEFT ARROW: Previous step")
    print("ESC: Exit")
    print("================\n")
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_SPACE:
                    paused = not paused
                    auto_play = not paused
                elif event.key == pygame.K_RIGHT:
                    if current_step < len(path) - 1:
                        current_step += 1
                    auto_play = False
                elif event.key == pygame.K_LEFT:
                    if current_step > 0:
                        current_step -= 1
                    auto_play = False
        
        # Auto-advance if not paused
        if auto_play and not paused:
            current_time = pygame.time.get_ticks()
            if current_time - last_step_time >= delay:
                if current_step < len(path) - 1:
                    current_step += 1
                    last_step_time = current_time
                else:
                    paused = True  # Pause when we reach the end
        
        # Draw current state
        draw_state(path[current_step], current_step, len(path))
        
        # Display completion message
        if current_step == len(path) - 1:
            success_text = header_font.render("SOLVED!", True, (0, 255, 0))
            screen.blit(success_text, (WINDOW_SIZE // 2 - 60, WINDOW_SIZE + HEADER_HEIGHT - 25))
            pygame.display.flip()
        
        clock.tick(60)  # 60 FPS
    
    pygame.quit()


## Simulation & Testing

In [36]:
# Visualize the solution
problem = (
    8, 6, 7,
    2, 5, 4,
    3, 0, 1
)

path = solve(problem)

if path:
    print(f"Solution found with {len(path) - 1} moves!")
    visualize_solution(path, delay=500)

Solution found with 31 moves!

=== Controls ===
SPACE: Pause/Resume
RIGHT ARROW: Next step
LEFT ARROW: Previous step
ESC: Exit

